# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [7]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-gzudfppv
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-gzudfppv
  Created wheel for Theano: filename=Theano-1.0.4+23.g630974a7b-cp37-none-any.whl size=2667498 sha256=2efaf59f9765d168d91f3f400533f3ff9481af6104aabbd85581aa397a9df892
  Stored in directory: /tmp/pip-ephem-wheel-cache-_sdkhy5m/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
     |████████████████████████████████| 86.3MB 300kB/s eta 0:00:012
     |████████████████████████████████| 51kB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 5.0MB/s  eta 0:00:01
     |████████████████████████████████| 2.4MB 16.4MB/s eta 0:00:01
     |████████████████████████████████| 450kB 9.4MB/s eta 0:00:01
     |████████████████████████████████| 112kB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 22.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 3

  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp37-none-any.whl size=61682 sha256=227c36534cb9211d72e4054ec66f01353f18d5fc6ca628f50aa78626395b3662
  Stored in directory: /home/jovyan/.cache/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4832 sha256=f70116b4bde81ea6c85cd6a4333de06d5b4174a1de2e9e92247027b6bd3563f7
  Stored in directory: /home/jovyan/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Created wheel for absl-py: filename=absl_py-0.8.1-cp37-none-any.whl size=121167 sha256=795e39d7be4abf8fe439f93912c7dcf3596757c4ee4645fa0a71f74ef4405f97
  Stored in directory: /home/jovyan/.cache/pip/wheels/a7/15/a0/0a0561549ad11cdc1bc8fa1191a353efd30facf6bfb507aefc
  Created wheel for wrapt: filename=wrapt-1.11.2-cp37-cp37m-linux_x86_64.whl size=70685 sha256=d5060d0f4c5a1800194e5bec5bc1806f7528fbf70dd265ef2c28ddb3bdab40e6
  Stored in directory: /home/

# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [9]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [10]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [13]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [14]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [ ]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 118us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 99us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 98us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 102us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 102us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 134us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 101us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 98us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 128us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Initiate activation function experimentation

# Initialising the 2nd ANN

In [14]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 242us/step - loss: 3.2881 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 2s 191us/step - loss: 3.2881 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 194us/step - loss: 3.2881 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 254us/step - loss: 3.2881 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 2s 218us/step - loss: 3.2881 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 187us/step - loss: 3.2881 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 185us/step - loss: 3.2881 - acc: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 180us/step - loss: 3.2881 - acc: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 183us/step - loss: 3.2881 - acc: 0.7960
Epoch 10/100
8000/8000 [==============================] - 2s 190us/step - loss: 3.2881 - acc: 0.7960

In [15]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initialising the 3rd ANN

In [17]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 242us/step - loss: 0.5123 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 2s 193us/step - loss: 0.4461 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 191us/step - loss: 0.4341 - acc: 0.7956
Epoch 4/100
8000/8000 [==============================] - 1s 187us/step - loss: 0.4297 - acc: 0.7982
Epoch 5/100
8000/8000 [==============================] - 2s 190us/step - loss: 0.4280 - acc: 0.8000
Epoch 6/100
8000/8000 [==============================] - 1s 186us/step - loss: 0.4276 - acc: 0.8030
Epoch 7/100
8000/8000 [==============================] - 2s 189us/step - loss: 0.4266 - acc: 0.8042
Epoch 8/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.4271 - acc: 0.8045
Epoch 9/100
8000/8000 [==============================] - 2s 252us/step - loss: 0.4268 - acc: 0.8049
Epoch 10/100
8000/8000 [==============================] - 2s 193us/step - loss: 0.4266 - acc: 0.8056

In [19]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1547   48]
 [ 265  140]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [ ]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)